In [1]:
from pyspark.sql import SparkSession
import json, os, re
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-spark_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

spark = SparkSession.builder \
    .appName("Json") \
    .master('spark://spark-master:7077') \
    .config('spark.executor.cores', '1') \
    .config('spark.executor.instances', '1') \
    .config('spark.driver.cores', '1') \
    .config('spark.cores.max', '3') \
    .config('spark.executor.memory', '512m') \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

file_path = "unique_realtor_data.json"  # Replace with your actual JSON file path
table_name = os.path.splitext(os.path.basename(file_path))[0]

df = spark.read.option("multiline", "true").json(file_path)

output_path = f"s3a://mysql/brozes/{table_name}"
print(output_path)
df.write.format("delta").mode("overwrite").save(output_path)

print("inserted successfully!")


s3a://mysql/brozes/unique_realtor_data
inserted successfully!
